In [1]:
from housing_pricer.scraping.sdk.data_manager import DataManager
from housing_pricer.data_processing.data_processing_utils import format_json_to_dataframe
import pandas as pd

pd.set_option('display.max_columns', None)
DATA_STORAGE = "../scraping/data_storage"

listings = format_json_to_dataframe(DataManager(DATA_STORAGE).load_data())
listings

Processing scraped JSON content to dataframe...: 132329it [00:05, 22084.09it/s]


,url_listing_type,url_listing_id,booli_id,sold_date,days_listed,residence_type,address,apartment_number,urban_area,municipality,construction_year,list_price,sold_price,sold_price_type,first_price,monthly_payment,rent,operating_cost,energy_class,floor,building_floors,latitude,longitude,has_solar_panels,agency_id,agent_id,booli_ids_of_previous_sales,n_previous_sales
0,bostad,2556516,5153109,2023-12-05,25.0,Villa,Västerbyvägen 9,None,Norsholm,Norrköping,1979.0,3325000.0,3250000.0,Slutpris,3 325 000,None,NaN,4398.0,D,None,NaN,58.505988,15.976788,None,26,11016,[5153109],1
1,bostad,1890105,5147734,2023-12-05,32.0,Villa,Sliparevägen 3,None,Rejmyre,Finspång,1978.0,1495000.0,1495000.0,Slutpris,1 495 000,None,NaN,4102.0,C,None,NaN,58.826413,15.938500,None,962,9543,"[5147734, -9035770]",2
2,bostad,1706785,5168064,2023-12-05,8.0,Villa,Kronoparksvägen 22,None,Ivetofta,Bromölla,1980.0,2095000.0,2400000.0,Slutpris,2 095 000,None,NaN,2648.0,C,None,NaN,56.091915,14.466163,None,20,11931,"[5168064, 984971, 350836]",3
3,bostad,3997519,5078126,2023-12-05,102.0,Lägenhet,Borgarfjordsgatan 21B,1303,Kista,Stockholm,2015.0,2495000.0,2450000.0,Slutpris,2 595 000,None,4274.0,NaN,D,3,NaN,59.404946,17.941505,None,16,6529,[5078126],1
4,annons,5119460,5119460,2023-12-05,39.0,Lägenhet,Rålambsvägen 48,None,Fredhäll,Stockholm,1931.0,3400000.0,3400000.0,Slutpris,3 195 000,None,2213.0,855.0,None,3,NaN,59.330127,18.005056,None,34,9162,[5119460],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131250,annons,4994336,4994336,2023-12-12,202.0,Lägenhet,Ringgatan 18B,None,Karls Grav,Vänersborg,1963.0,650000.0,600000.0,Slutpris,790 000,None,4161.0,173.0,None,1,NaN,58.365690,12.321370,None,34,4510,[4994336],1
131251,annons,5114004,5114004,2023-12-12,67.0,Lägenhet,Storgatan 70B,1004,Huvudsta,Solna,2015.0,4195000.0,4050000.0,Slutpris,3 995 000,None,5528.0,NaN,None,None,NaN,59.349895,17.985893,None,47,10439,[5114004],1
131252,annons,4928089,4928089,2023-12-12,271.0,Lägenhet,Vemdalsskalet Klockarstigen Norra 33,None,Klockarfjället,Härjedalen,2022.0,2195000.0,2075000.0,Slutpris,2 300 000,None,2271.0,1745.0,None,1,NaN,62.493445,13.932811,None,26,10717,[4928089],1
131253,bostad,4035039,5040108,2023-12-12,124.0,Lägenhet,Flyttblocksvägen 14C,1103,Bromma,Stockholm,1992.0,2550000.0,2500000.0,Slutpris,2 795 000,None,3874.0,500.0,D,2,NaN,59.358499,17.950154,None,21,13453,[5040108],1
